In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

#Setting up data
headernames = ["Surgery?","Age","HospitalNo","RectalTemp","Pulse","RespiratoryRate","ExtremitiesTemp","PeripheralPulse","MucousMembranes","CapillaryRefillTime","PainLevel","Peristalsis","AbdominalDistension","NasogastricTube","NasogastricReflux","NasogastricRefluxPH","Feces","Abdomen","PackedCellVolume","TotalProtein","AbdominocentesisAppearance","AbdominocentesisTotalProtein","Outcome","SurgicalLesion","LesionTypeOne","LesionTypeTwo","LesionTypeThree","CPData"]

print(len(headernames))

horsedataoriginal = pd.read_csv("horse-colic.data",sep=" ",index_col=False,names=headernames).replace('?', np.NaN) #change the data file path, also replace ? with NaN for easier process
horsedataoriginal = horsedataoriginal.iloc[:,:24]
horsedataoriginal

horsedataoriginal = horsedataoriginal.drop(columns=["HospitalNo"]) #Remove unused factors
horsedataoriginal = horsedataoriginal[0:]
horsedataoriginal["Outcome"] = horsedataoriginal["Outcome"].replace("3","2")
horsedataoriginal
# display(HTML(horsedataoriginal.to_html())) #To display all data

28


,Surgery?,Age,RectalTemp,Pulse,RespiratoryRate,ExtremitiesTemp,PeripheralPulse,MucousMembranes,CapillaryRefillTime,PainLevel,...,NasogastricReflux,NasogastricRefluxPH,Feces,Abdomen,PackedCellVolume,TotalProtein,AbdominocentesisAppearance,AbdominocentesisTotalProtein,Outcome,SurgicalLesion
0,2,1,38.50,66,28,3,3,NaN,2,5,...,NaN,NaN,3,5,45.00,8.40,NaN,NaN,2,2
1,1,1,39.2,88,20,NaN,NaN,4,1,3,...,NaN,NaN,4,2,50,85,2,2,2,2
2,2,1,38.30,40,24,1,1,3,1,3,...,NaN,NaN,1,1,33.00,6.70,NaN,NaN,1,2
3,1,9,39.10,164,84,4,1,6,2,2,...,2,5.00,3,NaN,48.00,7.20,3,5.30,2,1
4,2,1,37.30,104,35,NaN,NaN,6,2,NaN,...,NaN,NaN,NaN,NaN,74.00,7.40,NaN,NaN,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1,1,NaN,120,70,4,NaN,4,2,2,...,NaN,NaN,NaN,5,55,65,NaN,NaN,2,2
296,2,1,37.20,72,24,3,2,4,2,4,...,1,NaN,4,4,44.00,NaN,3,3.30,2,1
297,1,1,37.50,72,30,4,3,4,1,4,...,1,NaN,3,5,60.00,6.80,NaN,NaN,2,1
298,1,1,36.50,100,24,3,3,3,1,3,...,1,NaN,4,4,50.00,6.00,3,3.40,1,1


In [33]:
horsedata = horsedataoriginal
x_train = horsedata.loc[:,horsedata.columns!="Outcome"]
y_train = horsedata["Outcome"]
Kclassifier = KNeighborsClassifier()
Kclassifier.fit(x_train,y_train)

ValueError: Input X contains NaN.
KNeighborsClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values